### Imports

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Loading and Preparing the Data and Selecting the Right Columns
The aim is to use as many columns as necessary at first. We also want to include the following columns: borough, age, sex, ethnicity.

In [2]:
pas_1517 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_15_17.csv")
pas_1718 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_17_18.csv")
pas_1819 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_18_19.csv")
pas_1920 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_19_20.csv")
pas_2021 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_20_21.csv")

C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\1365595396.py:1: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,35,177,410,411,412,415,416,417,418,419,420,421,422,423) have mixed types. Specify dtype option on import or set low_memory=False.
  pas_1517 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_15_17.csv")
C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\1365595396.py:2: DtypeWarning: Columns (30,31,32,98,99,100,101,102,103,104,105,125,126,127,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  pas_1718 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_17_18.csv")
C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\1365595396.py:3: DtypeWarning: Columns (25,26,27,28,108,109,110,112,113,137,202,203,381) have mixed types. Specify dtype option on import or set low_memory=False.
  pas_1819 = pd.read_csv("pas_data_ward_level/PAS_ward_level_FY_18_19.csv")
C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\1365595396.py:4: DtypeW

In [3]:
# Making sure that the borough names are coherent
pas_1517 = pas_1517.rename(columns={'BOROUGHNEIGHBOURHOOD': 'Borough'})
pas_1718 = pas_1718.rename(columns={'BOROUGHNEIGHBOURHOOD': 'Borough'})
pas_1819 = pas_1819.rename(columns={'BOROUGHNEIGHBOURHOOD': 'Borough'})
pas_1920 = pas_1920.rename(columns={'BOROUGHNEIGHBOURHOOD': 'Borough'})

pas_1517['Borough'] = pas_1517['Borough'].str.split(' -').str[0]
pas_1718['Borough'] = pas_1718['Borough'].str.split(' -').str[0]
pas_1819['Borough'] = pas_1819['Borough'].str.split(' -').str[0]
pas_1920['Borough'] = pas_1920['Borough'].str.split(' -').str[0]

pas_1517 = pas_1517.rename(columns={'NQ147r': 'Ethnicity'})
pas_1517 = pas_1517.rename(columns={'XQ135r': 'Sex'})
pas_1517 = pas_1517.rename(columns={'Q136r': 'Age'})

pas_1718 = pas_1718.rename(columns={'NQ147r': 'Ethnicity'})
pas_1718 = pas_1718.rename(columns={'XQ135r': 'Sex'})
pas_1718 = pas_1718.rename(columns={'Q136r': 'Age'})

pas_1819 = pas_1819.rename(columns={'NQ147r': 'Ethnicity'})
pas_1819 = pas_1819.rename(columns={'XQ135r': 'Sex'})
pas_1819 = pas_1819.rename(columns={'Q136r': 'Age'})

pas_1920 = pas_1920.rename(columns={'NQ147r': 'Ethnicity'})
pas_1920 = pas_1920.rename(columns={'XQ135r': 'Sex'})
pas_1920 = pas_1920.rename(columns={'Q136r': 'Age'})

pas_2021 = pas_2021.rename(columns={'ReXQ135': 'Sex'})
pas_2021 = pas_2021.rename(columns={'ReQ136': 'Age'})
pas_2021 = pas_2021.rename(columns={'ReNQ147': 'Ethnicity'})

pas_2021['Borough'] = pas_2021['Borough'].str.replace('&', 'and', regex=False)

In [4]:
# Selecting all columns that are available in all of the datasets
columns_1517 = set(pas_1517.columns)
columns_1718 = set(pas_1718.columns)
columns_1819 = set(pas_1819.columns)
columns_1920 = set(pas_1920.columns)
columns_2021 = set(pas_2021.columns)

common_columns = list(columns_1517 & columns_1718 & columns_1819 & columns_1920 & columns_2021)

filtered_columns = [
    'A120', 'Q62TG', 'Q62A', 'Q61', 'Q62F', 'Q62C', 'Q62B',
    'Q13', 'RQ80E', 'Q133', 'Q15', 'Q148', 'Q3L', 'Q60', 'NQ135BD']

# ReXQ135 - sex (use one hot)
# ReQ136 - age (use one hot)
# ReNQ147 - ethnicity (use one hot)
# Borough (use one hot, do 1 analysis without)

# In age 20-21 change the word ' to ' to '-'
# In age rest change '65 or over' to '65+'

# NQ147r - ethnicity (15-20 all but the last)
# XQ135r - sex (-\\-)
# Q136 - age ()



additional_columns = ['Borough', 'Age', 'Sex', 'Ethnicity']

In [5]:
# Selecting the relevant columns from each dataset
pas_1517 = pas_1517[additional_columns + filtered_columns]
pas_1718 = pas_1718[additional_columns + filtered_columns]
pas_1819 = pas_1819[additional_columns + filtered_columns]
pas_1920 = pas_1920[additional_columns + filtered_columns]
pas_2021 = pas_2021[additional_columns + filtered_columns]

# Formating age
pas_2021['Age'] = pas_2021['Age'].str.replace(" to ", "-")
pas_1517['Age'] = pas_1517['Age'].str.replace("65 or over", "65+")
pas_1718['Age'] = pas_1718['Age'].str.replace("65 or over", "65+")
pas_1819['Age'] = pas_1819['Age'].str.replace("65 or over", "65+")
pas_1920['Age'] = pas_1920['Age'].str.replace("65 or over", "65+")

pas_df = pd.concat([pas_1517, pas_1718, pas_1819, pas_1920, pas_2021], ignore_index=True)

In [6]:
pas_df_cleaned = pas_df.dropna()
nan_counts = pas_df_cleaned.isna().sum()

pas_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60022 entries, 1 to 76397
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Borough    60022 non-null  object
 1   Age        60022 non-null  object
 2   Sex        60022 non-null  object
 3   Ethnicity  60022 non-null  object
 4   A120       60022 non-null  object
 5   Q62TG      60022 non-null  object
 6   Q62A       60022 non-null  object
 7   Q61        60022 non-null  object
 8   Q62F       60022 non-null  object
 9   Q62C       60022 non-null  object
 10  Q62B       60022 non-null  object
 11  Q13        60022 non-null  object
 12  RQ80E      60022 non-null  object
 13  Q133       60022 non-null  object
 14  Q15        60022 non-null  object
 15  Q148       60022 non-null  object
 16  Q3L        60022 non-null  object
 17  Q60        60022 non-null  object
 18  NQ135BD    60022 non-null  object
dtypes: object(19)
memory usage: 9.2+ MB


In [7]:
# scorings

scoring_1 = {'Excellent': 4, 'Good': 3, 'Fair': 2, 'Poor': 1, 'Very poor': 0}
scoring_2 = {'Definitely agree': 3, 'Tend to agree': 2, 'Tend to disagree': 1, 'Definitely disagree': 0}
scoring_3 = {'Strongly agree': 4, 'Tend to agree': 3, 'Neither agree nor disagree': 2, 'Tend to disagree': 1, 'Strongly disagree': 0}
scoring_4 = {'At least daily': 5, 'At least weekly': 4, 'At least fortnightly': 3, 'At least monthly': 2, 'Less often': 1, 'Never': 0}
scoring_5 = {'Very worried': 3, 'Fairly worried': 2, 'Not very worried': 1, 'Not at all worried': 0}
scoring_6 = {'Yes': 1, 'No': 0}
scoring_7 = {'Very well informed': 2, 'Fairly well informed': 1, 'Not at all informed': 0}

In [8]:
# pas_df_cleaned['Q144'] = pas_df_cleaned['Q144'].map(scoring_1)
pas_df_cleaned.loc[:, 'A120'] = pas_df_cleaned['A120'].map(scoring_3)
pas_df_cleaned.loc[:, 'Q62TG'] = pas_df_cleaned['Q62TG'].map(scoring_3)
pas_df_cleaned.loc[:, 'Q62A'] = pas_df_cleaned['Q62A'].map(scoring_3)
pas_df_cleaned.loc[:, 'Q61'] = pas_df_cleaned['Q61'].map(scoring_1)
# pas_df_cleaned['Q131'] = pas_df_cleaned['Q131'].map(scoring_4)
# pas_df_cleaned['NQ133A'] = pas_df_cleaned['NQ133A'].map(scoring_2)
pas_df_cleaned.loc[:, 'Q62F'] = pas_df_cleaned['Q62F'].map(scoring_3)
# pas_df_cleaned['Q3F'] = pas_df_cleaned['Q3F'].map(scoring_2)
pas_df_cleaned.loc[:, 'Q62C'] = pas_df_cleaned['Q62C'].map(scoring_3)
# pas_df_cleaned['Q1'] = pas_df_cleaned['Q1'].map(scoring_2)
# pas_df_cleaned['Q3C'] = pas_df_cleaned['Q3C'].map(scoring_2)
# pas_df_cleaned['Q3J'] = pas_df_cleaned['Q3J'].map(scoring_2)
pas_df_cleaned.loc[:, 'Q62B'] = pas_df_cleaned['Q62B'].map(scoring_3)
# pas_df_cleaned['BQ90A'] = pas_df_cleaned['BQ90A'].map(scoring_4) don't know
pas_df_cleaned.loc[:, 'Q13'] = pas_df_cleaned['Q13'].map(scoring_5)
pas_df_cleaned.loc[:, 'RQ80E'] = pas_df_cleaned['RQ80E'].map(scoring_6)
pas_df_cleaned.loc[:, 'Q133'] = pas_df_cleaned['Q133'].map(scoring_7)
pas_df_cleaned.loc[:, 'Q15'] = pas_df_cleaned['Q15'].map(scoring_5)
pas_df_cleaned.loc[:, 'Q148'] = pas_df_cleaned['Q148'].map(scoring_6) # Disability
# pas_df_cleaned['NQ62B'] = pas_df_cleaned['NQ62B'].map(scoring_3)
pas_df_cleaned.loc[:, 'Q3L'] = pas_df_cleaned['Q3L'].map(scoring_2)
# pas_df_cleaned['NQ143'] = pas_df_cleaned['NQ143'].map(scoring_4)
pas_df_cleaned.loc[:, 'Q60'] = pas_df_cleaned['Q60'].map(scoring_1)
pas_df_cleaned.loc[:, 'NQ135BD'] = pas_df_cleaned['NQ135BD'].map(scoring_3)

In [9]:
pas_df_cleaned

,Borough,Age,Sex,Ethnicity,A120,Q62TG,Q62A,Q61,Q62F,Q62C,Q62B,Q13,RQ80E,Q133,Q15,Q148,Q3L,Q60,NQ135BD
1,Kensington and Chelsea,25-34,Female,White British,3,3,2,3,3,3,3,2,0,1,1,0,1,3,4
3,Kensington and Chelsea,16-24,Male,Asian,2,3,4,3,3,4,4,0,1,1,0,0,2,3,4
4,Kensington and Chelsea,35-44,Male,White British,0,3,3,3,3,3,3,1,0,0,1,0,2,3,4
5,Kensington and Chelsea,55-64,Female,White Other,3,3,3,3,3,3,3,1,0,0,1,1,2,3,3
6,Kensington and Chelsea,65+,Female,White British,4,4,4,3,4,4,4,1,0,0,1,0,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76392,Kensington and Chelsea,55-64,Female,Other,2,3,4,3,3,4,4,1,0,0,2,0,3,3,4
76393,Kensington and Chelsea,35-44,Male,Other,0,3,2,2,3,3,3,2,0,1,1,0,3,2,3
76394,Kensington and Chelsea,25-34,Female,Black,4,3,3,3,3,3,3,0,1,2,2,0,2,3,4
76396,Kensington and Chelsea,45-54,Female,Mixed,3,4,4,4,4,3,4,1,1,0,1,0,2,3,4


In [10]:
nan_counts = pas_df_cleaned.isna().sum()
nan_counts

Borough      0
Age          0
Sex          0
Ethnicity    0
A120         0
Q62TG        0
Q62A         0
Q61          0
Q62F         0
Q62C         0
Q62B         0
Q13          0
RQ80E        0
Q133         0
Q15          0
Q148         0
Q3L          0
Q60          0
NQ135BD      0
dtype: int64

In [11]:
# Doing one hot encoding 

age_dummies = pd.get_dummies(pas_df_cleaned['Age'], prefix='Age', dtype=int)
sex_dummies = pd.get_dummies(pas_df_cleaned['Sex'], prefix='Sex', dtype=int)
ethnicity_dummies = pd.get_dummies(pas_df_cleaned['Ethnicity'], prefix='Ethnicity', dtype=int)

pas_df_cleaned.drop(['Age', 'Sex', 'Ethnicity', 'Borough'], axis=1, inplace=True)

df_pas = pd.concat([pas_df_cleaned, age_dummies, sex_dummies, ethnicity_dummies], axis=1)


C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\2309912597.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pas_df_cleaned.drop(['Age', 'Sex', 'Ethnicity', 'Borough'], axis=1, inplace=True)


In [12]:
df_pas

,A120,Q62TG,Q62A,Q61,Q62F,Q62C,Q62B,Q13,RQ80E,Q133,...,Sex_Female,Sex_Male,Sex_Other,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_Refused,Ethnicity_White British,Ethnicity_White Other
1,3,3,2,3,3,3,3,2,0,1,...,1,0,0,0,0,0,0,0,1,0
3,2,3,4,3,3,4,4,0,1,1,...,0,1,0,1,0,0,0,0,0,0
4,0,3,3,3,3,3,3,1,0,0,...,0,1,0,0,0,0,0,0,1,0
5,3,3,3,3,3,3,3,1,0,0,...,1,0,0,0,0,0,0,0,0,1
6,4,4,4,3,4,4,4,1,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76392,2,3,4,3,3,4,4,1,0,0,...,1,0,0,0,0,0,1,0,0,0
76393,0,3,2,2,3,3,3,2,0,1,...,0,1,0,0,0,0,1,0,0,0
76394,4,3,3,3,3,3,3,0,1,2,...,1,0,0,0,1,0,0,0,0,0
76396,3,4,4,4,4,3,4,1,1,0,...,1,0,0,0,0,1,0,0,0,0


In [13]:
df_pas = df_pas[df_pas['NQ135BD'] != 2]  # Remove rows where the value is 2
df_pas['NQ135BD'] = df_pas['NQ135BD'].apply(lambda x: 0 if x in [0, 1] else 1)

# Check the distribution of the modified target variable
print('Modified target distribution:')
print(np.bincount(df_pas['NQ135BD']))

Modified target distribution:
[ 3196 51787]


C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\779449902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pas['NQ135BD'] = df_pas['NQ135BD'].apply(lambda x: 0 if x in [0, 1] else 1)


In [14]:
# Splitting the dataset
X = df_pas.drop('NQ135BD', axis=1)
y = df_pas['NQ135BD']

# I researched how to split the data into 3 with this link: https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Convert the target variable to integer type
y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)


### For later, the dataset is not very balanced, take care of that

In [15]:
print('Training set target distribution')
print(np.bincount(y_train))

Training set target distribution
[ 1962 31027]


In [16]:
correlation_matrix = df_pas.corr()
threshold = 0.70
high_corr_var = np.where(np.abs(correlation_matrix) > threshold)
high_corr_var = [(correlation_matrix.index[x], correlation_matrix.columns[y]) 
                 for x, y in zip(*high_corr_var) 
                 if x != y and x < y]

features_to_drop = set()

for (feature1, feature2) in high_corr_var:
    features_to_drop.add(feature2)

X_train_reduced = X_train.drop(columns=list(features_to_drop))
X_val_reduced = X_val.drop(columns=list(features_to_drop))
X_test_reduced = X_test.drop(columns=list(features_to_drop))

# Scaling the data
scaler = StandardScaler()
X_train_reduced_scaled = scaler.fit_transform(X_train_reduced)
X_val_reduced_scaled = scaler.transform(X_val_reduced)
X_test_reduced_scaled = scaler.transform(X_test_reduced)

## The Model

### Grid Search to find the best parameters

In [17]:
# logreg = LogisticRegression() 

# param_grid = [
#     {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l2'], 'multi_class': ['ovr', 'multinomial'], 'solver': ['lbfgs', 'saga']},
#     {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1'], 'multi_class': ['ovr'], 'solver': ['liblinear', 'saga']}
# ]

# # Set up the GridSearchCV
# grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='f1', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train_reduced_scaled, y_train)

# # Get the best parameters and the best model
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# print(f"Best parameters found: {best_params}")

# # Evaluate the best model on the validation set
# y_val_pred = best_model.predict(X_val_reduced_scaled)
# val_accuracy = accuracy_score(y_val, y_val_pred)
# print(f"Validation accuracy: {val_accuracy}")

# print("Classification report on validation set:")
# print(classification_report(y_val, y_val_pred))

# # Evaluate the best model on the test set
# y_test_pred = best_model.predict(X_test_reduced_scaled)
# test_accuracy = accuracy_score(y_test, y_test_pred)
# print(f"Test accuracy: {test_accuracy}")

# print("Classification report on test set:")
# print(classification_report(y_test, y_test_pred))

# 'C': 0.01, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'saga'

In [18]:
logreg = LogisticRegression(C=0.01, multi_class='ovr', penalty='l2', solver='saga', max_iter=100000000)

# Fit the model to the training data
logreg.fit(X_train_reduced_scaled, y_train)

# Evaluate the model on the validation set
y_val_pred = logreg.predict(X_val_reduced_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy: {val_accuracy}")

print("Classification report on validation set:")
print(classification_report(y_val, y_val_pred))

# Evaluate the model on the test set
y_test_pred = logreg.predict(X_test_reduced_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test accuracy: {test_accuracy}")

print("Classification report on test set:")
print(classification_report(y_test, y_test_pred))

Validation accuracy: 0.963808311357643
Classification report on validation set:
              precision    recall  f1-score   support

           0       0.75      0.48      0.58       583
           1       0.97      0.99      0.98     10414

    accuracy                           0.96     10997
   macro avg       0.86      0.73      0.78     10997
weighted avg       0.96      0.96      0.96     10997

Test accuracy: 0.9595344184777667
Classification report on test set:
              precision    recall  f1-score   support

           0       0.76      0.46      0.57       651
           1       0.97      0.99      0.98     10346

    accuracy                           0.96     10997
   macro avg       0.87      0.72      0.78     10997
weighted avg       0.95      0.96      0.95     10997



In [19]:
from sklearn.metrics import confusion_matrix


# Test set Confusion Matrix
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("Confusion matrix for test set:")
print(conf_matrix_test)

# Coefficients
coefficients = logreg.coef_
print("Coefficients of the logistic regression model:")
print(coefficients)

# Extract the coefficients of the logistic regression model
feature_names = X_train_reduced.columns
coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': logreg.coef_[0]  # Since we have binary classification, there is only one set of coefficients
})

# Sort the DataFrame by the absolute value of the coefficients for better readability
coefficients_df['Absolute Coefficient'] = coefficients_df['Coefficient'].abs()
coefficients_df = coefficients_df.sort_values(by='Absolute Coefficient', ascending=False).drop('Absolute Coefficient', axis=1)

print(coefficients_df)


Confusion matrix for test set:
[[  298   353]
 [   92 10254]]
Coefficients of the logistic regression model:
[[ 0.29188457  0.31698572  0.37019105  0.60474368  0.43593847  0.30381905
  -0.04070985 -0.03749153  0.0647547   0.05758224 -0.12832752  0.07490014
   0.35628333 -0.01935241 -0.04905314  0.01794811 -0.00119779  0.00746878
   0.04066257 -0.03908132  0.07264939 -0.03801168  0.07202208 -0.24146007
  -0.09770877  0.00981411 -0.06182092  0.07669239  0.07217403]]
                    Feature  Coefficient
3                       Q61     0.604744
4                      Q62C     0.435938
2                      Q62A     0.370191
12                      Q60     0.356283
1                     Q62TG     0.316986
5                      Q62B     0.303819
0                      A120     0.291885
23          Ethnicity_Black    -0.241460
10                     Q148    -0.128328
24          Ethnicity_Mixed    -0.097709
27  Ethnicity_White British     0.076692
11                      Q3L     0.07490

### Statsmodels for statistical signifficance

In [20]:
X_train_const = sm.add_constant(X_train_reduced_scaled)
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit(maxiter=1000)
summary_table = result.summary2().tables[1]

Optimization terminated successfully.
         Current function value: 0.116752
         Iterations 351


In [21]:
significant_coefficients = summary_table[summary_table['P>|z|'] < 0.05]

print('Statistically Signifficant coefficients')
print(significant_coefficients)

Statistically Signifficant coefficients
        Coef.  Std.Err.          z         P>|z|    [0.025    0.975]
x1   0.311596  0.027387  11.377667  5.402594e-30  0.257919  0.365272
x2   0.322602  0.031739  10.164226  2.863879e-24  0.260395  0.384809
x3   0.387871  0.031819  12.189840  3.521129e-34  0.325507  0.450236
x4   0.650429  0.032773  19.846237  1.187685e-87  0.586194  0.714664
x5   0.454698  0.030131  15.090546  1.868893e-51  0.395642  0.513754
x6   0.305917  0.028353  10.789748  3.848420e-27  0.250347  0.361487
x10  0.072294  0.035403   2.042048  4.114673e-02  0.002906  0.141683
x11 -0.142583  0.031069  -4.589222  4.449011e-06 -0.203478 -0.081689
x12  0.078641  0.029760   2.642486  8.229991e-03  0.020312  0.136970
x13  0.361411  0.035669  10.132459  3.965501e-24  0.291501  0.431320
x21  0.082773  0.031826   2.600790  9.300940e-03  0.020395  0.145151
x22 -0.039434  0.018618  -2.118029  3.417261e-02 -0.075925 -0.002943


In [27]:
# Get feature names
feature_names = X_train_reduced.columns
feature_names = ['const'] + list(feature_names)

coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient_statsmodels': result.params,
    'P-Value': result.pvalues
})

# Only signifficant under 95% CI
significant_coefficients_df = coefficients_df[coefficients_df['P-Value'] < 0.05]
significant_coefficients_df['Absolute Coefficient'] = significant_coefficients_df['Coefficient_statsmodels'].abs()
significant_coefficients_df = significant_coefficients_df.sort_values(by='Absolute Coefficient', ascending=False).drop('Absolute Coefficient', axis=1)

# Get coefficients from the more advanced model and add them to the stats models summary table
logreg_coefficients = logreg.coef_[0]
significant_coefficients_df = significant_coefficients_df.reset_index(drop=True)
logreg_coefficients_df = pd.DataFrame({
    'Feature': feature_names[1:],
    'Coefficient_scikit': logreg_coefficients
})
merged_df = pd.merge(significant_coefficients_df, logreg_coefficients_df, on='Feature', how='left')
merged_df['P-Value'] = merged_df['P-Value'].apply(lambda x: f'{x:.4f}')
#
print('Statistically Significant Coefficients:')
print(merged_df)

Statistically Significant Coefficients:
       Feature  Coefficient_statsmodels P-Value  Coefficient_scikit
0          Q61                 0.650429  0.0000            0.604744
1         Q62C                 0.454698  0.0000            0.435938
2         Q62A                 0.387871  0.0000            0.370191
3          Q60                 0.361411  0.0000            0.356283
4        Q62TG                 0.322602  0.0000            0.316986
5         A120                 0.311596  0.0000            0.291885
6         Q62B                 0.305917  0.0000            0.303819
7         Q148                -0.142583  0.0000           -0.128328
8   Sex_Female                 0.082773  0.0093            0.072649
9          Q3L                 0.078641  0.0082            0.074900
10         Q15                 0.072294  0.0411            0.057582
11   Sex_Other                -0.039434  0.0342           -0.038012


C:\Users\20220900\AppData\Local\Temp\ipykernel_27440\2085553541.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  significant_coefficients_df['Absolute Coefficient'] = significant_coefficients_df['Coefficient_statsmodels'].abs()


In [29]:
# Calculate odds ratios
merged_df['Odds_Ratio_statsmodels'] = np.exp(merged_df['Coefficient_statsmodels'])
merged_df['Odds_Ratio_scikit'] = np.exp(merged_df['Coefficient_scikit'])


print('Statistically Significant Coefficients with Odds Ratios:')
print(merged_df)


Statistically Significant Coefficients with Odds Ratios:
       Feature  Coefficient_statsmodels P-Value  Coefficient_scikit   
0          Q61                 0.650429  0.0000            0.604744  \
1         Q62C                 0.454698  0.0000            0.435938   
2         Q62A                 0.387871  0.0000            0.370191   
3          Q60                 0.361411  0.0000            0.356283   
4        Q62TG                 0.322602  0.0000            0.316986   
5         A120                 0.311596  0.0000            0.291885   
6         Q62B                 0.305917  0.0000            0.303819   
7         Q148                -0.142583  0.0000           -0.128328   
8   Sex_Female                 0.082773  0.0093            0.072649   
9          Q3L                 0.078641  0.0082            0.074900   
10         Q15                 0.072294  0.0411            0.057582   
11   Sex_Other                -0.039434  0.0342           -0.038012   

    Odds_Ratio_stat